<img src='https://raw.githubusercontent.com/autonomio/hyperio/master/logo.png' width=250px>

# Predicting Diabetes with Keras

### A Very Short Introduction to Hyperparameter Optimization with Talos

The goal of a Talos experiment, is to find a set of suitable hyperparameters for Keras model. In order to do this, you need to have three things: 

- Keras model 
- Talos hyperparameter dictionary
- Talos experiment configuration

Below we will briefly overview each.

In [0]:
!pip install talos

     |████████████████████████████████| 26.6MB 7.4MB/s 
     |████████████████████████████████| 839kB 35.8MB/s 
  Created wheel for talos: filename=talos-0.6.3-cp36-none-any.whl size=49626 sha256=ac66838616140e4bef4e2fa2154c60131ed34f9c726ec14a4afb348044aa7811
  Stored in directory: /root/.cache/pip/wheels/bb/d7/6b/86fd8b1fc7cfbd2c54796412f86efb5fb6a3a5c734014f6a66
  Created wheel for wrangle: filename=wrangle-0.6.7-cp36-none-any.whl size=49894 sha256=48e38728d291851c5c482e5dcba0074bab903f73920fd5e9073a4a325a71621d
  Stored in directory: /root/.cache/pip/wheels/bf/1b/50/d0403ce6ef269e364894da7b50db68db14c4ac62c577561e2d
  Created wheel for astetik: filename=astetik-1.9.9-cp36-none-any.whl size=56960 sha256=1d1a4388e64149904159c46437b5a3b771c76d1bd2443839e921b6f377949879
  Stored in directory: /root/.cache/pip/wheels/ae/70/21/c475cd079ec401dd6e1b9b1d42b4c38554ce12679bfb214aad
  Created wheel for chances: filename=chances-0.1.9-cp36-none-any.whl size=41609 sha256=f54f8e8fc02ca60b3ab1b838

### The Keras Model

As a model, any Keras model will do. Let's consider as an example a very simple model that makes a prediction on the classic *Pima Indians Diabetes* dataset. A brief overview of the dataset can be found [here](https://www.kaggle.com/uciml/pima-indians-diabetes-database) and the dataset we will use can be found [here](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv). The below model does not require you to separately download the file.

In [0]:
from numpy import loadtxt

dataset = loadtxt("https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv", delimiter=",")
x = dataset[:,0:8]
y = dataset[:, 8]

In [0]:
from keras.models import Sequential
from keras.layers import Dense

def diabetes():
    
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X, Y, epochs=100, batch_size=10, verbose=0)
    
    return model

Using TensorFlow backend.


### Talos Hyperparameter Dictionary

Let's prepare for an experiment where we will optimize against three common attributes:

- neurons on the first layer
- activations
- batch_sizes

In [0]:
from keras.activations import relu, elu

p = {
    'first_neuron': [12, 24, 48],
    'activation': ['relu', 'elu'],
    'batch_size': [10, 20, 30]
}

### Configuring the Keras Model for Talos

In order to prepare a Keras model for a Talos experiment, we need to do four things:

- add input parameters to the function
- replace the hyperparameter inputs with references to params dictionary
- make sure model.fit() stores the history object
- modify the output of the model

These steps are always the same.

In [0]:
# add input parameters to the function
def diabetes(x_train, y_train, x_val, y_val, params):
    print("\nParams:",params)
    # replace the hyperparameter inputs with references to params dictionary 
    model = Sequential()
    model.add(Dense(params['first_neuron'], input_dim=8, activation=params['activation']))
    #model.add(Dense(8, activation=params['activation']))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # make sure history object is returned by model.fit()
    out = model.fit(x=x, 
                    y=y,
                    validation_data=[x_val, y_val],
                    epochs=100,
                    batch_size=params['batch_size'],
                    verbose=0)
    
    # modify the output model
    return out, model

That's it, there is nothing more to it. A more complicated experiment would just entail more of the same in terms of the way the params dictionary references are made. Otherwise the changes would always be exactly the same.

### Talos Experiment

The Talos experiment is performed through the Scan() command. In case you don't have Talos installed already, you can do that now.

In [0]:
import talos

While many configurations are possible, the only things that you absolutely must input to a Talos experiment are: 

- x
- y
- params (the dictionary 'p' we created above)
- model (the 'diabetes' we created above)

In [10]:
t = talos.Scan(x=x, y=y, params=p, model=diabetes, experiment_name='diabetes')


  0%|          | 0/18 [00:00<?, ?it/s]

Params: {'activation': 'relu', 'batch_size': 10, 'first_neuron': 12}



  6%|▌         | 1/18 [00:34<09:49, 34.66s/it]

Params: {'activation': 'relu', 'batch_size': 10, 'first_neuron': 24}



 11%|█         | 2/18 [01:09<09:14, 34.67s/it]

...


 89%|████████▉ | 16/18 [06:01<00:37, 18.64s/it]

Params: {'activation': 'elu', 'batch_size': 30, 'first_neuron': 24}



 94%|█████████▍| 17/18 [06:14<00:16, 16.74s/it]

Params: {'activation': 'elu', 'batch_size': 30, 'first_neuron': 48}



100%|██████████| 18/18 [06:26<00:00, 15.27s/it]
